#Задание  
По аналогии с седьмым заданием, исходные данные оттуда

In [ ]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python
!pip install --upgrade scikit-learn==0.23.0

In [46]:
import cv2
import zipfile
import numpy as np
import pandas as pd
from imutils import paths
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
with zipfile.ZipFile('train_task.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [6]:
imagePaths = sorted(list(paths.list_images('train')))

In [7]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [13]:
X = []
y = []
for image in imagePaths:
  X.append(extract_histogram(cv2.imread(image)))
  if 'cat' in image:
    y.append(1)
  else:
    y.append(0)

In [14]:
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.242645,0.073122,0.000232,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.087363,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.418818,0.126781,0.000085,0.000000,0.000000,0.0,0.0,0.0,0.005085,0.076441,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000042,0.000000
2,0.356737,0.184168,0.020616,0.001619,0.000717,0.0,0.0,0.0,0.000041,0.027440,...,0.00002,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.007726
3,0.001266,0.019995,0.011941,0.001583,0.000158,0.0,0.0,0.0,0.000000,0.002145,...,0.01595,0.002233,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.023354,0.497077
4,0.000000,0.000105,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000366,...,0.00000,0.000000,0.0,0.002719,0.7748,0.34338,0.038583,0.114442,0.126833,0.008469


Классификатор с мягким зазором

In [72]:
lsvc = LinearSVC(C = 1.75, random_state = 195)

Бэггинг деревьев принятия решений

In [73]:
dtc = DecisionTreeClassifier(criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 195)

In [74]:
bagging_dtc = BaggingClassifier(dtc, n_estimators = 12, random_state = 195)

Случайный лес

In [75]:
rfc = RandomForestClassifier(n_estimators = 12, criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 195)

Логическая регрессия

In [76]:
lr = LogisticRegression(solver='lbfgs', random_state = 195)

Метаалгоритм

In [83]:
estimators = [('lsvc', lsvc), ('rfc', rfc), ('b_dtc', bagging_dtc)]

In [84]:
sc = StackingClassifier(estimators, final_estimator=lr, cv = 2).fit(X, y)
sc.classes_

array([0, 1])

**Доля правильной классификации**

In [85]:
sc.score(X, y)

0.83

**Вероятность отнесения изображений к классу 1 с помощью обученного метаалгоритма**

In [80]:
round(sc.predict_proba([extract_histogram(cv2.imread('dog.1032.jpg'))])[0, 1], 3)

0.672

In [81]:
round(sc.predict_proba([extract_histogram(cv2.imread('cat.1048.jpg'))])[0, 1], 3)

0.381

In [82]:
round(sc.predict_proba([extract_histogram(cv2.imread('dog.1024.jpg'))])[0, 1], 3)

0.528

In [61]:
round(sc.predict_proba([extract_histogram(cv2.imread('cat.1016.jpg'))])[0, 1], 3)

0.743